### Khởi tạo trễ
- Cho tới nay, ta đã làm những điều không trực quan sau:
    1. Ta định nghĩa kiến trúc mạng mà không xét đến chiều đầu vào.
    2. Ta thêm tầng mà không xét đến chiều đầu ra của tầng trước đó.
    3. Ta thậm chí còn khởi tạo các tham số mà không có đầy đủ thông tin để xác định số lượng các tham số của mô hình.
- Suy cho cùng, MXNet hay tất cả các framework khác đều không thể dự đoán được chiều của đầu vào. Thủ thuật ở đây đó là MXNet đã khởi tạo trễ, tức là đợi cho đến khi ta truyền dữ liệu qua mô hình lần đều để suy ra kích thước của mỗi tầng khi chúng di chuyển.
- Sau này, khi làm việc với các mạng nơ ron tích chập, kỹ thuật này sẽ còn trở nên tiện lợi hơn bởi vì chiều của đầu vào (tức độ phân giải của bức ảnh) sẽ tác động đến chiều của các tầng tiếp theo trong mạng. Do đó, khả năng gán giá trị các tham số mà không cần biết số chiều tại thời điển viết mã có thể đơn giản hóa việc xác định và sửa đổi mô hình về sau.

#### 1. Khởi tạo một mạng

In [7]:
from mxnet import init, np, npx
from mxnet.gluon import nn
npx.set_np()

In [8]:
def getnet():
    net = nn.Sequential()
    net.add(nn.Dense(256, activation='relu'))
    net.add(nn.Dense(10))
    return net

net = getnet()

- Lúc này mạng nơ ron không thể biết được chiều của các trọng số ở tầng đầu vào bởi nó còn chưa biết chiều của đầu vào, và vì thế MXNet chưa thể khởi tạo bất kỳ tham số nào cả. Ta có thể xác thực việc này bằng cách truy cập các tham số như dưới đây.

In [9]:
print(net.collect_params)
print(net.collect_params())

<bound method Block.collect_params of Sequential(
  (0): Dense(-1 -> 256, Activation(relu))
  (1): Dense(-1 -> 10, linear)
)>
sequential1_ (
  Parameter dense2_weight (shape=(256, -1), dtype=float32)
  Parameter dense2_bias (shape=(256,), dtype=float32)
  Parameter dense3_weight (shape=(10, -1), dtype=float32)
  Parameter dense3_bias (shape=(10,), dtype=float32)
)


Chú ý rằng mặc dù đối tượng Parameter có tồn tại, chiều đầu vào của mỗi tầng được liệt kê là -1. MXNet sử dụng giá trị đặc biệt -1 để ám chỉ việc chưa biết chiều tham số. Tại thời điểm này, việc thử truy cập __net[0].weight.data()__ sẽ gây ra lỗi thực thi báo rằng mạng cần khởi tạo trước khi truy cập tham số.

In [10]:
net.initialize()
net.collect_params()

sequential1_ (
  Parameter dense2_weight (shape=(256, -1), dtype=float32)
  Parameter dense2_bias (shape=(256,), dtype=float32)
  Parameter dense3_weight (shape=(10, -1), dtype=float32)
  Parameter dense3_bias (shape=(10,), dtype=float32)
)

- Như đã thấy, không có gì thay đổi ở đây cả, khi chưa biết chiều của đầu vào, việc gọi phương thức khởi tạo không thực sự khởi tạo các tham số. Thay vào đó, việc gọi phương thức trên sẽ chỉ đăng ký với MXNet là chúng ta muốn khởi tạo các tham số và phân phối mà ta muốn dùng để khởi tạo. Chỉ khi truyền dữ liệu qua mạng thì MXNet mới khởi tạo các tham số và ta mới thấy được sự khác biệt.

In [11]:
x = np.random.uniform(size = (2, 20))
net(x)

net.collect_params()

sequential1_ (
  Parameter dense2_weight (shape=(256, 20), dtype=float32)
  Parameter dense2_bias (shape=(256,), dtype=float32)
  Parameter dense3_weight (shape=(10, 256), dtype=float32)
  Parameter dense3_bias (shape=(10,), dtype=float32)
)

Ngay khi biết được chiều của đầu vào là (2, 20) MXNet có thể xác định kích thước của ma trận trọng số tầng đầu tiên W1 là (256, 20). Sau khi biết được kích thước tầng đầu tiên thì tính kích thước tầng thứ hai và cứ thế đi hết đồ thị tính toán cho đến khi biết đượch hết kích thước của mọi tầng.
- Chú ý rằng trong trường hợp này, chỉ tầng đầu tiên cần được khởi tạo trễ nhưng MXNet vẫn khởi tạo theo thứ tự. Khi mà tất cả kích thước tham số đã được biết, MXNet cuối cùng có thể thực hiện khởi tạo các tham số.

#### 2. Khởi tạo trễ trong thực tiễn
- Trong mục này ta sẽ xem thử khi nào thì việc khởi tạo thực sự diễn ra. Ta sẽ tạo một bộ khởi tạo, nó sẽ không làm gì ngoài việc gửi một thông điệp gỡ lỗi cho biết khi nào nó được gọi và cùng với các tham số nào.

In [16]:
class MyInit(init.Initializer):
    def _init_weight(self, name, data):
        print("init", name, data.shape)

        data[:] = np.random.uniform(size = data.shape)

net = getnet()
net.initialize(init = MyInit())

Lưu ý rằng mặc dù MyInit sẽ in thông tin về các tham số của mô hình sau khi nó được gọi, hàm khởi tạo Initialize không xuất bất cứ thông tin nào được thực thi, do đó viêc khởi tạo tham số không thực sự thực hiện khi gọi hàm initialize. Kế tiếp, ta định nghĩa đầu vào và thực hiện một phép truyền xuôi.

In [17]:
x = np.random.uniform(size = (2, 20))
y = net(x)

net.collect_params()

init dense8_weight (256, 20)
init dense9_weight (10, 256)


sequential4_ (
  Parameter dense8_weight (shape=(256, 20), dtype=float32)
  Parameter dense8_bias (shape=(256,), dtype=float32)
  Parameter dense9_weight (shape=(10, 256), dtype=float32)
  Parameter dense9_bias (shape=(10,), dtype=float32)
)

- Lúc này, thông tin về các tham số mô hình mới được in ra. Khi thực hiện lượt truyền xuôi dựa trên biến đầu vào x, hệ thống có thể tự động suy ra kích thước các tham số của tất cả các tầng dựa trên kích thước của biến đầu vào này..
- Một khi hệ thống đã tạo ra các tham số trên, nó sẽ gọi thực thể MyInit để khởi tạo chúng trước khi bắt đầu thực hiện lượt truyền xuôi.

- Việc khởi tạo này sẽ chỉ được gọi khi lượt truyền xuôi đầu tiên hoàn thành. Sau thời điểm này, chúng ta sẽ không khởi tạo lại khi dùng lệnh net(x) nữa.

#### 3. Khởi tạo cưỡng chế
- Khởi tạo trễ không xảy ra nếu hệ thống đã biết trước kích thước của tất cả các tham số khi gọi hàm __initialize__. Việc này có thể xảy ra trong hai trường hợp:
    1. Ta đã truyền dữ liệu qua mạng từ trước và chỉ muốn khởi tạo lại các tham số
    2. Ta đã chỉ rõ cả chiều đầu vào và chiều đầu ra của mạng khi định nghĩa nó.


In [19]:
net.initialize(init = MyInit(), force_reinit=True)
net.initialize(init = MyInit())

init dense8_weight (256, 20)
init dense9_weight (10, 256)


d:\Anaconda\envs\d2l\lib\site-packages\mxnet\gluon\parameter.py:887: UserWarning: Parameter 'dense8_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
d:\Anaconda\envs\d2l\lib\site-packages\mxnet\gluon\parameter.py:887: UserWarning: Parameter 'dense8_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
d:\Anaconda\envs\d2l\lib\site-packages\mxnet\gluon\parameter.py:887: UserWarning: Parameter 'dense9_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
d:\Anaconda\envs\d2l\lib\site-packages\mxnet\gluon\parameter.py:887: UserWarning: Parameter 'dense9_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)


In [20]:
net = nn.Sequential()
net.add(nn.Dense(256, in_units=20, activation='relu'))
net.add(nn.Dense(10, in_units=256))

net.initialize(init = MyInit())

init dense10_weight (256, 20)
init dense11_weight (10, 256)
